In [ ]:
'''
from cvzone import HandTrackingModule
import cv2
import numpy as np
import os
from tkinter import Tk, filedialog

def exponential_smoothing(current_point, prev_point, alpha=0.2):
    if prev_point is None:
        return current_point
    else:
        return tuple(map(int, np.multiply(current_point, alpha) + np.multiply(prev_point, 1 - alpha)))

def calculate_velocity(current_point, prev_point, time_interval=1):
    if prev_point is not None:
        displacement = np.linalg.norm(np.array(current_point) - np.array(prev_point))
        velocity = displacement / time_interval
        return velocity
    else:
        return 0

def detect_hands():
    cap = cv2.VideoCapture(0)
    screen_width, screen_height = 1920, 1080
    cap.set(3, screen_width)
    cap.set(4, screen_height)

    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    hand_detector = HandTrackingModule.HandDetector(detectionCon=0.6, maxHands=1)

    small_corner_width, small_corner_height = 320, 240
    cv2.namedWindow('Canvas with Small Video Corner', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Canvas with Small Video Corner', screen_width, screen_height)

    canvas = None
    drawn_paths = []

    drawing = False
    prev_smoothed_point = None

    def open_folder(event, x, y, flags, param):
        nonlocal canvas
        if event == cv2.EVENT_LBUTTONDOWN and 10 < x < 200 and 10 < y < 50:
            folder_path = filedialog.askdirectory(title="Select Folder")
            if folder_path:
                open_image_dialog(folder_path)

    def open_image_dialog(folder_path):
        nonlocal canvas
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(initialdir=folder_path, title="Select Image",
                                               filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        root.destroy()

        if file_path:
            canvas = cv2.imread(file_path)
            drawn_paths.clear()

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)
        canvas_resized = cv2.resize(canvas, (frame.shape[1], frame.shape[0])) if canvas is not None else np.zeros_like(frame)

        hands, _ = hand_detector.findHands(frame)

        if hands:
            hand = hands[0]
            index_finger_tip = hand['lmList'][8][:2]
            middle_finger_tip = hand['lmList'][12][:2]
            thumb_tip = hand['lmList'][4][:2]

            if np.linalg.norm(np.array(index_finger_tip) - np.array(thumb_tip)) < 40:
                drawn_paths = []
                drawing = False
                prev_smoothed_point = None

            fingers_distance = np.linalg.norm(np.array(index_finger_tip) - np.array(middle_finger_tip))
            if fingers_distance < 30:  # Adjusted threshold value
                drawing = False
                prev_smoothed_point = None
            else:
                drawing = True

            if (
                thumb_tip[1] < index_finger_tip[1] < middle_finger_tip[1]
                and thumb_tip[0] > index_finger_tip[0] > middle_finger_tip[0]
            ):
                canvas = None  # Reset canvas when making a "V" shape

            if drawing:
                smoothed_point = exponential_smoothing(index_finger_tip, prev_smoothed_point, alpha=0.1)
                hand_velocity = calculate_velocity(index_finger_tip, prev_smoothed_point)

                if hand_velocity > 10:
                    pass

                if prev_smoothed_point:
                    cv2.line(canvas_resized, prev_smoothed_point, smoothed_point, (255, 255, 255), 5)
                    drawn_paths.append((prev_smoothed_point, smoothed_point))
                prev_smoothed_point = smoothed_point

        canvas_with_small_video = np.copy(canvas_resized)
        small_corner_frame = cv2.resize(frame, (small_corner_width, small_corner_height))
        canvas_with_small_video[10:10 + small_corner_height, 10:10 + small_corner_width] = small_corner_frame

        for path in drawn_paths:
            cv2.line(canvas_with_small_video, path[0], path[1], (255, 255, 255), 5)

        if drawing:
             cv2.circle(canvas_with_small_video, smoothed_point, 5, (255, 255, 255), -1)
        # Add the button
        button_text = "Open Folder"
        cv2.rectangle(canvas_with_small_video, (10, 10), (200, 50), (0, 255, 0), -1)
        cv2.putText(canvas_with_small_video, button_text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Set the mouse callback function
        cv2.setMouseCallback('Canvas with Small Video Corner', open_folder)

        cv2.imshow('Canvas with Small Video Corner', canvas_with_small_video)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            snapshot_filename = "drawn_snapshot.png"
            cv2.imwrite(snapshot_filename, canvas_resized)
            print(f"Snapshot saved as {snapshot_filename}")

        if key & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_hands()
'''

In [1]:
from cvzone import HandTrackingModule
import cv2
import numpy as np
from tkinter import Tk, filedialog

def exponential_smoothing(current_point, prev_point, alpha=0.2):
    if prev_point is None:
        return current_point
    else:
        return tuple(map(int, np.multiply(current_point, alpha) + np.multiply(prev_point, 1 - alpha)))

def calculate_velocity(current_point, prev_point, time_interval=1):
    if prev_point is not None:
        displacement = np.linalg.norm(np.array(current_point) - np.array(prev_point))
        velocity = displacement / time_interval
        return velocity
    else:
        return 0

def detect_hands():
    cap = cv2.VideoCapture(0)
    screen_width, screen_height = 1920, 1080
    cap.set(3, screen_width)
    cap.set(4, screen_height)

    
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    hand_detector = HandTrackingModule.HandDetector(detectionCon=0.6, maxHands=1)

    small_corner_width, small_corner_height = 320, 240
    cv2.namedWindow('Canvas with Small Video Corner', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Canvas with Small Video Corner', screen_width, screen_height)

    canvas = None
    drawn_paths = []

    drawing = False
    prev_smoothed_point = None

    def open_folder(event, x, y, flags, param):
        nonlocal canvas
        if event == cv2.EVENT_LBUTTONDOWN and 10 < x < 200 and 10 < y < 50:
            folder_path = filedialog.askdirectory(title="Select Folder")
            if folder_path:
                open_image_dialog(folder_path)

    def open_image_dialog(folder_path):
        nonlocal canvas
        root = Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename(initialdir=folder_path, title="Select Image",
                                               filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
        root.destroy()

        if file_path:
            canvas = cv2.imread(file_path)
            drawn_paths.clear()

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)
        canvas_resized = cv2.resize(canvas, (frame.shape[1], frame.shape[0])) if canvas is not None else np.zeros_like(frame)

        hands, _ = hand_detector.findHands(frame)

        if hands:
            hand = hands[0]
            index_finger_tip = hand['lmList'][8][:2]
            middle_finger_tip = hand['lmList'][12][:2]
            thumb_tip = hand['lmList'][4][:2]

            if np.linalg.norm(np.array(index_finger_tip) - np.array(thumb_tip)) < 40:
                drawn_paths = []
                drawing = False
                prev_smoothed_point = None

            fingers_distance = np.linalg.norm(np.array(index_finger_tip) - np.array(middle_finger_tip))
            if fingers_distance < 30:  # Adjusted threshold value
                drawing = False
                prev_smoothed_point = None
            else:
                drawing = True

            if (
                thumb_tip[1] < index_finger_tip[1] < middle_finger_tip[1]
                and thumb_tip[0] > index_finger_tip[0] > middle_finger_tip[0]
            ):
                canvas = None  # Reset canvas when making a "V" shape

            if drawing:
                smoothed_point = exponential_smoothing(index_finger_tip, prev_smoothed_point, alpha=0.1)
                hand_velocity = calculate_velocity(index_finger_tip, prev_smoothed_point)

                if hand_velocity > 10:
                    pass

                if prev_smoothed_point:
                    cv2.line(canvas_resized, prev_smoothed_point, smoothed_point, (255, 255, 255), 5)
                    drawn_paths.append((prev_smoothed_point, smoothed_point))
                prev_smoothed_point = smoothed_point

            # Check if the hand is closed
            if all(hand['lmList'][i][1] > hand['lmList'][i + 1][1] for i in range(0, len(hand['lmList']) - 1, 4)):

                print("Hand closed👍. Program exited.")
                break  # Break out of the loop to exit the program

        canvas_with_small_video = np.copy(canvas_resized)
        small_corner_frame = cv2.resize(frame, (small_corner_width, small_corner_height))
        canvas_with_small_video[10:10 + small_corner_height, 10:10 + small_corner_width] = small_corner_frame

        for path in drawn_paths:
            cv2.line(canvas_with_small_video, path[0], path[1], (255, 255, 255), 5)

        if drawing:
            cv2.circle(canvas_with_small_video, smoothed_point, 5, (255, 255, 255), -1)

        # Add the button
        button_text = "Open Image"
        cv2.rectangle(canvas_with_small_video, (10, 10), (200, 50), (0, 255, 0), -1)
        cv2.putText(canvas_with_small_video, button_text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

        # Set the mouse callback function
        cv2.setMouseCallback('Canvas with Small Video Corner', open_folder)

        cv2.imshow('Canvas with Small Video Corner', canvas_with_small_video)

        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):
            snapshot_filename = "drawn_snapshot.png"
            cv2.imwrite(snapshot_filename, canvas_resized)
            print(f"Snapshot saved as {snapshot_filename}")

        if key & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_hands()


Hand closed👍. Program exited.
